In [1]:
# Import all packages and dependencies
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import sys, warnings, os
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from IPython.core.display import HTML, display

In [2]:
#Setting columns and rows to display all the results
pd.set_option("display.max_columns", None, "display.max_rows", None)

In [3]:
def load_notebook_config(width=True):
    """
    Loads all neccesary configuration for the notebook's style:
     - plots styling.
     - pandas table sizes and limiting amount of float decimals.
     - adjust the notebook cells width
    """
    pd.options.display.max_columns = 0
    pd.set_option('display.float_format', lambda x: '%.4f' % x)
    pd.options.mode.chained_assignment = None

    if width:
        display(HTML("""<link href='https://fonts.googleapis.com/css?family=Montserrat' rel='stylesheet'>
                        <style> div.text_cell_render{font-family: 'Montserrat';}
                                .container { width:95% !important;}
                        </style>"""))
load_notebook_config()

In [4]:
#Ignore warnings when validating scores
if not sys.warnoptions:
    warnings.simplefilter("ignore")
    os.environ["PYTHONWARNINGS"] = "ignore"

In [5]:
# Read in the dataset
df = pd.read_csv("../Resources/data_preprocessed.csv")
df.head()

,index,Winner,title_bout,lose_streak_dif,win_streak_dif,longest_win_streak_dif,win_dif,loss_dif,total_round_dif,total_title_bout_dif,ko_dif,sub_dif,height_dif,reach_dif,age_dif,sig_str_dif,avg_sub_att_dif,avg_td_dif,draw_diff,avg_sig_str_pct_diff,avg_TD_pct_diff,win_by_Decision_Majority_diff,win_by_Decision_Split_diff,win_by_Decision_Unanimous_diff,win_by_TKO_Doctor_Stoppage_diff,odds_diff,ev_diff,kd_bout_diff,sig_str_landed_bout_diff,sig_str_attempted_bout_diff,sig_str_pct_bout_diff,tot_str_landed_bout_diff,tot_str_attempted_bout_diff,td_landed_bout_diff,td_attempted_bout_diff,td_pct_bout_diff,sub_attempts_bout_diff,pass_bout_diff,rev_bout_diff,Stance_diff,better_rank_enc,level_0
0,0,0,0,0,-1,-7,-27,-13,-57,-1,-17,-8,7.6200,0.0000,-8,1.0500,-0.6000,-0.7500,0,0.0000,0.0000,-1,1,-2,0,-332,-95.0549,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0,-1,0
1,1,1,0,0,0,0,12,7,80,9,2,1,-12.7000,-5.0800,11,-3.1800,-0.2000,1.2100,1,0.0000,0.0000,0,0,9,0,700,275.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,2,-1,1
2,2,1,0,-1,0,-3,-6,-3,-21,0,-2,-2,0.0000,2.5400,-3,-4.2000,-1.1000,-1.0800,0,0.0000,0.0000,0,-1,-1,0,225,20.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,-1,-1,2
3,3,0,0,0,-1,-1,5,2,15,0,0,3,2.5400,-5.0800,-5,-1.0900,0.1000,0.7100,1,0.0000,0.0000,0,1,1,0,230,25.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,-1,-1,3
4,4,0,0,-1,0,0,6,3,29,0,-2,4,-7.6200,-7.6200,5,-1.8200,0.6000,2.8600,0,0.0000,0.0000,0,2,2,0,400,130.5556,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1,0,4


In [6]:
# Index is only needed for SQL join so we must drop it here so it doesnt effect outcomes
index = ['index', 'level_0']
df.drop(index, axis=1, inplace = True)

In [7]:
# Establish X variables and y target
y = df.Winner
X = df.drop(['Winner'], axis=1)

In [8]:
# Split dataset into train test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size = 0.25)

In [9]:
# Creating a StandardScaler instance.
scaler = StandardScaler()
# Fitting the Standard Scaler with the training data.
X_scaler = scaler.fit(X_train)

# Scaling the data.
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [10]:
rf_model = RandomForestClassifier(random_state=42)

In [11]:
# Fit the model w the training data
rf_model.fit(X_train_scaled, y_train)

RandomForestClassifier(random_state=42)

In [12]:
# Making predictions using the testing data.
predictions = rf_model.predict(X_test_scaled)
acc_score = accuracy_score(y_test, predictions)

In [13]:
# Calculating the confusion matrix.
cm = confusion_matrix(y_test, predictions)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

cm_df

,Predicted 0,Predicted 1
Actual 0,310,148
Actual 1,105,579


In [14]:
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,310,148
Actual 1,105,579


Accuracy Score : 0.7784588441330998
Classification Report
              precision    recall  f1-score   support

           0       0.75      0.68      0.71       458
           1       0.80      0.85      0.82       684

    accuracy                           0.78      1142
   macro avg       0.77      0.76      0.77      1142
weighted avg       0.78      0.78      0.78      1142



In [15]:
# Calculate feature importance in the Random Forest model.
importances = rf_model.feature_importances_
importances

array([0.00276661, 0.01342351, 0.01540536, 0.01933018, 0.02084814,
       0.0229023 , 0.03097715, 0.01001945, 0.01686059, 0.01555615,
       0.02620706, 0.03053441, 0.03312924, 0.03981271, 0.03136694,
       0.03869132, 0.00244197, 0.02433504, 0.        , 0.00280308,
       0.01154955, 0.01713814, 0.00313078, 0.06649627, 0.07322299,
       0.02589769, 0.09371279, 0.03185838, 0.04309731, 0.05467295,
       0.04129331, 0.02245628, 0.01210351, 0.02278072, 0.02191735,
       0.03792105, 0.00351165, 0.01442107, 0.00540704])

In [16]:
# We can sort the features by their importance.
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

[(0.09371278924756822, 'sig_str_landed_bout_diff'),
 (0.07322299039122629, 'ev_diff'),
 (0.06649627355115442, 'odds_diff'),
 (0.05467294529755924, 'tot_str_landed_bout_diff'),
 (0.04309730671872242, 'sig_str_pct_bout_diff'),
 (0.041293308302487015, 'tot_str_attempted_bout_diff'),
 (0.03981271071059609, 'sig_str_dif'),
 (0.038691324197812044, 'avg_td_dif'),
 (0.03792105149725804, 'pass_bout_diff'),
 (0.03312924027637572, 'age_dif'),
 (0.0318583831513803, 'sig_str_attempted_bout_diff'),
 (0.031366939727302395, 'avg_sub_att_dif'),
 (0.030977152153706743, 'total_round_dif'),
 (0.03053441075092353, 'reach_dif'),
 (0.026207058330942615, 'height_dif'),
 (0.025897685861511234, 'kd_bout_diff'),
 (0.024335036416857133, 'avg_sig_str_pct_diff'),
 (0.02290230179793294, 'loss_dif'),
 (0.022780723959308514, 'td_pct_bout_diff'),
 (0.02245627665762514, 'td_landed_bout_diff'),
 (0.021917345103222123, 'sub_attempts_bout_diff'),
 (0.020848136817350923, 'win_dif'),
 (0.019330181830940606, 'longest_win_stre

In [17]:
# Define the model - deep neural net
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  25
hidden_nodes_layer2 = 12

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))


# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=25)

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/25
107/107 [==============================] - 1s 1ms/step - loss: 0.6136 - accuracy: 0.6524
Epoch 2/25
107/107 [==============================] - 0s 931us/step - loss: 0.4674 - accuracy: 0.7746
Epoch 3/25
107/107 [==============================] - 0s 1ms/step - loss: 0.4511 - accuracy: 0.7697
Epoch 4/25
107/107 [==============================] - 0s 1ms/step - loss: 0.4418 - accuracy: 0.7748
Epoch 5/25
107/107 [==============================] - 0s 2ms/step - loss: 0.4318 - accuracy: 0.7806
Epoch 6/25
107/107 [==============================] - 0s 1ms/step - loss: 0.4212 - accuracy: 0.7946
Epoch 7/25
107/107 [==============================] - 0s 2ms/step - loss: 0.4132 - accuracy: 0.7960
Epoch 8/25
107/107 [==============================] - 0s 2ms/step - loss: 0.4127 - accuracy: 0.7958
Epoch 9/25
107/107 [==============================] - 0s 1ms/step - loss: 0.4142 - accuracy: 0.7934
Epoch 10/25
107/107 [==============================] - 0s 1ms/step - loss: 0.4163 - accuracy: 0.79